In [10]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from itertools import product

path_raw_data = '../data/raw/'
path_processed_data = '../data/processed/'

In [11]:
list_target = [
    'ig', 
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
    ]
list_splits = [
    'train', 
    'test'
    ]
list_datasets = [
    'top_mentioned_timelines',
    'users'
]

In [19]:
possible_comb = list(product(list_target, list_splits, list_datasets))

In [23]:

for target, split, dataset in tqdm(possible_comb, total = len(possible_comb)):
    
    
    if dataset == 'top_mentioned_timelines':
        path_data_input = path_raw_data + f'{split}_r3_{target}_top_mentioned_timelines.csv'
        path_output_base = path_processed_data + f'{split}_r3_{target}_top_mentioned_timelines_separated_comments'
        text_col = 'Texts'
    if dataset == 'users':
        path_data_input = path_raw_data + f'r3_{target}_{split}_users.csv'
        path_output_base = path_processed_data + f'r3_{target}_{split}_users_separated_comments'
        text_col = 'Timeline'

    data = pd.read_csv(
        path_data_input, 
        sep = ';', 
        encoding='utf-8-sig'
        )
    
    # Separa os textos em linhas individuais
    df_sep_comments = data.assign(Texts=data[text_col].str.split(' # ')).explode(text_col)

    # Reindexa o DataFrame resultante
    df_sep_comments.reset_index(drop=True, inplace = True)

    df_sep_comments.ffill(inplace = True)
    
    df_sep_comments[text_col] = df_sep_comments[text_col].apply(str.lower)
    
    df_sep_comments['id_text'], unique = pd.factorize(df_sep_comments[text_col])
    
    df_sep_comments_unique = df_sep_comments.drop_duplicates('id_text')
    
    df_sep_comments.to_csv(path_output_base + '.csv', index = False)
    df_sep_comments_unique.to_csv(path_output_base + '_unique.csv', index = False)


100%|██████████| 24/24 [07:21<00:00, 18.38s/it]
